# Medical Appointment No Shows Capstone

# 5. MODELING

In [4]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [8]:
path = 'data/step2_output.csv'
df = pd.read_csv(path, index_col=None) 
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,NoShow
0,29872499824296,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,558997776694438,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4262962299951,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,867951213174,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8841186448183,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,No


## 1. LINEAR REGRESSION MODEL

In [ ]:
from sklearn.linear_model import LinearRegression
linreg_model = LinearRegression()
model.fit(X,y)
model.predict(X)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_predicted, y_test))

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# Use stock symbols to extract training data
X = all_prices[['ebay','nvideia','yahoo']]
y = all_prices[['apple']]

# Fit and score the model with cross-validation
scores = cross_val_score(Ridge(),X,y, cv=3)
print(scores)

## 2. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42).fit(X_train, y_train)
model.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrics(y_test,model.predict(X_test))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [10,200],
    'max_features': ['auto','sqrt','log2',0.5],
}

gs = GridSearchCV(estimator=model, param_grid=param_grid,\
                 scoring='recall',n_jobs=-1)
gs.fit(X_train,y_train)
recall_score(y_test,gs.best_estimator_.predict(X_test))

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, \
    precision_recall_curve, auc
import seaborn as sns
%matplotlib inline 
%config InlineBackend.figure_format = 'svg'
X,y = make_classification(n_samples=10000, n_classes=2, \
                         weights =[0.95,0.05],random_state=42)
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=.2,\
                                                  random_state=2)
model = LogisticRegression().fit(X_train,y_train)

probs = model.predict_proba(X_test)
probs = probs[:,1]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train, targets,\
                                              test_size=0.2, \
                                              random_state=42)
model=RandomForestRegressor(max_features=.5,n_estimators=20)
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
from sklearn.svm import LinearSVC

# Construct data for the model
X = df[['Age', 'Neighborhood']]
y = df[['Age']]

model = LinearSVC() 
#Fit X(Training Data), y(Labels for each datapoint)
model.fit(X,y)

#Investigate what kind of patterns the data found
model.coef_

# Predict with the model
predictions = model.predict(X_test)
print(predictions)

# Create input array
X_predict = targets[['petal length (cm)', 'petal width (cm)']]



# Visualize predictions and actual values
plt.scatter(X_predict['petal length (cm)'], X_predict['petal width (cm)'],
            c=predictions, cmap=plt.cm.coolwarm)
plt.title("Predicted class values")
plt.show()

In [ ]:
from sklearn import linear_model

# Prepare input and output DataFrames
X = boston[['AGE']]
y = boston[['RM']]

# Fit the model
model = linear_model.LinearRegression()
model.fit(X,y)


# Generate predictions with the model using those inputs
predictions = model.predict(new_inputs.reshape([-1,1]))

# Visualize the inputs and predicted values
plt.scatter(new_inputs, predictions, color='r', s=3)
plt.xlabel('inputs')
plt.ylabel('predictions')
plt.show()